In [16]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime as dt

In [17]:
key = pd.read_csv('key.csv')
weather = pd.read_csv('weather.csv', parse_dates=['date'])
train = pd.read_csv('train.csv', parse_dates=['date'])

In [18]:
train = pd.merge(train,key)

In [19]:
rt = train[train['units'] > 0].sort_values(by=['units'], ascending=False).reset_index(drop=True)

In [20]:
# weahter: codesum split & get dummies

# extract codsum unique values
cs_iter = (set(x.split(' ')) for x in weather.codesum)
cs = sorted(set.union(*cs_iter))

dummies = pd.DataFrame(np.zeros((len(weather), len(cs))), columns = cs)

for i, gen in enumerate(weather.codesum):
    dummies.ix[i, gen.split(' ')] = 1
    
weather_cs = weather.join(dummies)
del weather_cs['codesum']
del weather_cs['']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


In [21]:
df = pd.merge(rt, weather_cs, on=['date', 'station_nbr'])

In [22]:
# 
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [27]:
df.[depart]

Index(['date', 'store_nbr', 'item_nbr', 'units', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', 'BCFG', 'BLDU', 'BLSN', 'BR',
       'DU', 'DZ', 'FG', 'FG+', 'FU', 'FZDZ', 'FZFG', 'FZRA', 'GR', 'GS', 'HZ',
       'MIFG', 'PL', 'PRFG', 'RA', 'SG', 'SN', 'SQ', 'TS', 'TSRA', 'TSSN',
       'UP', 'VCFG', 'VCTS', 'year', 'month', 'day'],
      dtype='object')

In [54]:
# counting M

columns = list(df.columns)

def count_M(columns):
    ls_M_counting = []
    ls_M_columns= []
    ls_all_counting = []
    
    for i in columns:
        a = df[i].value_counts()
        b = df[i].name
        
        if 'M' in a:
            ls_M_counting.append(a['M'])
            ls_M_columns.append(b)
            ls_all_counting.append(a.sum())
    return ls_M_counting, ls_M_columns, ls_all_counting

        
    
k = count_M(columns)
k = np.array(k).T

M_columns = ['M', 'column', 'total']
dfdf = pd.DataFrame(data = k, columns = M_columns)
dfdf['M'] = dfdf['M'].astype(int)
dfdf['total'] = dfdf['total'].astype(int)
dfdf['Pct'] = dfdf['M'] / dfdf['total'] * 100
dfdf.sort_values(by='Pct', ascending=False)

,M,column,total,Pct
3,79710,depart,118696,67.154748
8,42900,snowfall,118696,36.142751
11,4957,sealevel,118696,4.176215
2,4520,tavg,118696,3.808047
6,4520,heat,118696,3.808047
7,4520,cool,118696,3.808047
5,4492,wetbulb,118696,3.784458
0,3018,tmax,118696,2.542630
1,3016,tmin,118696,2.540945
10,2987,stnpressure,118696,2.516513


In [55]:
# M -> numeric

for col in df.columns:
    if df[col].dtypes == 'object':
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else: continue

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118696 entries, 0 to 118695
Data columns (total 53 columns):
date           118696 non-null datetime64[ns]
store_nbr      118696 non-null int64
item_nbr       118696 non-null int64
units          118696 non-null int64
station_nbr    118696 non-null int64
tmax           115678 non-null float64
tmin           115680 non-null float64
tavg           114176 non-null float64
depart         38986 non-null float64
dewpoint       116125 non-null float64
wetbulb        114204 non-null float64
heat           114176 non-null float64
cool           114176 non-null float64
sunrise        49541 non-null float64
sunset         49541 non-null float64
snowfall       74553 non-null float64
preciptotal    101316 non-null float64
stnpressure    115709 non-null float64
sealevel       113739 non-null float64
resultspeed    116506 non-null float64
resultdir      116506 non-null float64
avgspeed       115937 non-null float64
BCFG           118696 non-null float